<a href="https://colab.research.google.com/github/Pikilman/thesis/blob/main/EVALUATING_ROBUST_RETRIEVAL_PIPELINES.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# QUERY GENERATION
## Cloning G. Penha repository to get access to disentangled_information_needs and other query generation functions.

In [ ]:
!git clone https://github.com/Guzpenha/query_variation_generators.git

Cloning into 'query_variation_generators'...
remote: Enumerating objects: 320, done.
remote: Counting objects: 100% (320/320), done.
remote: Compressing objects: 100% (207/207), done.
remote: Total 320 (delta 209), reused 215 (delta 110), pack-reused 0
Receiving objects: 100% (320/320), 165.85 KiB | 4.61 MiB/s, done.
Resolving deltas: 100% (209/209), done.


In [ ]:
from google.colab import drive

drive.mount('/content/gdrive/', force_remount=True)

Mounted at /content/gdrive/


#QUERY SUMMARY
Counting valid queries and their transformation type, displaying in table for each set of annoted queries. Author queries are available in G. Penha repository as .csv files at '/content/query_variation_generators/data/variations_trec2019_labeled.csv' and '/content/query_variation_generators/data/variations_antique_labeled.csv'

In [ ]:
# Importing necessary dataframe and expression counting functions
import pandas as pd
import re
#reading in files of annoted query variations from Penha, G et al.
file_n_antique= "/content/query_variation_generators/data/variations_antique_labeled.csv"
file_n_trec = "/content/query_variation_generators/data/variations_trec2019_labeled.csv"
query_variations_trec = pd.read_csv(file_n_trec)
query_variations_antiques = pd.read_csv(file_n_antique)
#Counting variations for methods and variation type for TREC2019 labelled set.
query_variations_trec["query"] = query_variations_trec.apply(lambda r, re=re: re.sub('[\W_]', ' ',  r['original_query'].lower()), axis=1)
query_variations_trec["variation"] = query_variations_trec.apply(lambda r, re=re: re.sub('[\W_]', ' ',  r['variation'].lower()), axis=1)
query_variations_trec["variation"] = query_variations_trec.apply(lambda r: r['query'] if r['variation'].strip() == "" else r['variation'], axis=1)
query_variations_trec['qid'] = query_variations_trec['q_id'].astype(str)
#Counting variations for methods and variation type for ANTIQUE labelled set.
query_variations_antiques["query"] = query_variations_antiques.apply(lambda r, re=re: re.sub('[\W_]', ' ',  r['original_query'].lower()), axis=1)
query_variations_antiques["variation"] = query_variations_antiques.apply(lambda r, re=re: re.sub('[\W_]', ' ',  r['variation'].lower()), axis=1)
query_variations_antiques["variation"] = query_variations_antiques.apply(lambda r: r['query'] if r['variation'].strip() == "" else r['variation'], axis=1)
query_variations_antiques['qid'] = query_variations_antiques['q_id'].astype(str)

In [ ]:
#Displaying Dataframe table for TREC2019 variation counts as performed by Penha, G et al.
query_variations_trec = query_variations_trec[(query_variations_trec["valid"])].groupby(["transformation_type","method"]).count().reset_index()[["transformation_type", "method", "dataset"]]
query_variations_trec.columns = ["transformation_type", "method", "count"]
query_variations_trec

,transformation_type,method,count
0,mispelling,WordSwapNeighboringCharacterSwap,43
1,mispelling,WordSwapQWERTY,42
2,mispelling,WordSwapRandomCharacterSubstitution,42
3,naturality,naturality_by_removing_stop_words,37
4,naturality,summarization_with_t5-base_from_description_to...,35
5,ordering,WordInnerSwapRandom,43
6,paraphrase,back_translation_pivot_language_de,23
7,paraphrase,ramsrigouthamg/t5_paraphraser,26
8,synonym,WordSwapEmbedding,27
9,synonym,WordSwapWordNet,16


Displaying DATAFRAME table for ANTIQUE dataset query variation counts as performed by Penha, G et al.

In [ ]:
#Displaying Dataframe table for ANTIQUE variation counts as performed by Penha, G et al.
query_variations_antiques = query_variations_antiques[(query_variations_antiques["valid"])].groupby(["transformation_type","method"]).count().reset_index()[["transformation_type", "method", "dataset"]]
query_variations_antiques.columns = ["transformation_type", "method", "count"]
query_variations_antiques

,transformation_type,method,count
0,mispelling,WordSwapNeighboringCharacterSwap,199
1,mispelling,WordSwapQWERTY,182
2,mispelling,WordSwapRandomCharacterSubstitution,197
3,naturality,naturality_by_removing_stop_words,199
4,naturality,summarization_with_t5-base_from_description_to...,136
5,ordering,WordInnerSwapRandom,200
6,paraphrase,back_translation_pivot_language_de,93
7,paraphrase,ramsrigouthamg/t5_paraphraser,105
8,synonym,WordSwapEmbedding,124
9,synonym,WordSwapWordNet,71


These .csv files contain the same original queries and their respective variations used in the authors' paper. To generate the provided queries methods provided by the authors can be used.
## Generating Query Variations
##### Necessarily need to access the authors' github repository to make use of the authors' query variation generators. Additional requirements are also imported and installed. The methods implemented by Penha, G et al. can then be replicated.

Launching a python virtual environment and installing necessary packages

In [ ]:
!git clone https://github.com/Guzpenha/query_variation_generators.git
%cd /content/query_variation_generators
!apt-get install python3-venv
!python3 -m venv env
!source env/bin/activate
!pip install -e .
!pip install torch transformers IPython pandas ir_datasets textattack protobuf python-terrier trectools
!pip install git+https://github.com/Georgetown-IR-Lab/OpenNIR git+https://github.com/terrierteam/pyterrier_doc2query.git
!pip install git+https://github.com/terrierteam/pyterrier_t5

fatal: destination path 'query_variation_generators' already exists and is not an empty directory.
/content/query_variation_generators
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  python3-pip-whl python3-setuptools-whl python3.10-venv
The following NEW packages will be installed:
  python3-pip-whl python3-setuptools-whl python3-venv python3.10-venv
0 upgraded, 4 newly installed, 0 to remove and 19 not upgraded.
Need to get 2,474 kB of archives.
After this operation, 2,888 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 python3-pip-whl all 22.0.2+dfsg-1ubuntu0.3 [1,679 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 python3-setuptools-whl all 59.6.0-1.2ubuntu0.22.04.1 [788 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 python3.10-venv amd64 3.10.12-1~22.04.2 [5,724 B]
Get:

Generating Query Variations

In [ ]:
from disentangled_information_needs.transformations.synonym import SynonymActions
from disentangled_information_needs.transformations.paraphrase import ParaphraseActions
from disentangled_information_needs.transformations.naturality import NaturalityActions
from disentangled_information_needs.transformations.mispelling import MispellingActions
from disentangled_information_needs.transformations.ordering import OrderingActions

import pandas as pd
import ir_datasets
import nltk
nltk.download('omw-1.4')

textattack: Updating TextAttack package dependencies.
textattack: Downloading NLTK required packages.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package omw to /root/nltk_data...
[nltk_data] Downloading package universal_tagset to /root/nltk_data...
[nltk_data]   Unzipping taggers/universal_tagset.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
textattack: Downloading https://textattack.s3.amazonaws.com/word_embeddings/paragramcf.
100%|██████████| 481M/481M [00:15<00:00, 31.7MB/s]
textattack: Unzipping file /root/.cache/textattack/tmpb_lyvs8c.zip to /root/.cache/textattack/word_embeddings/paragramcf.
textattack: Su

True

Setting training parameters for desc2t5 query variations and paraphrasing tasks

In [ ]:
#task = 'car/v1.5/train/fold0'
task ='antique/train/split200-valid'
out_dir = "."
sample = 1000

Using 'ir_datasets' package to access data set and 'disentangled_information_needs' to perform appropriate transformations

In [ ]:
dataset = ir_datasets.load(task)
queries = [t[1].lower() for t in dataset.queries_iter()]
q_ids = [t[0] for t in dataset.queries_iter()]

pa = ParaphraseActions(queries, q_ids, out_dir)
transformed_queries_paraphrase_models = pa.seq2seq_paraphrase(sample=sample)
transformed_queries_back_translation = pa.back_translation_paraphrase(sample=sample)

na = NaturalityActions(queries, q_ids)
#transformed_queries_trec_desc_to_title = na.naturality_by_trec_desc_to_title(model_path="", sample=sample)
transformed_queries_stop_word_removal = na.remove_stop_words(sample=sample)
transformed_queries_stop_word_and_stratified_removal = na.remove_stop_words_and_stratify_by_len(sample=sample)
transformed_queries_summarizer = na.naturality_by_summarization(sample=sample)

sa = SynonymActions(queries, q_ids)
transformed_queries_syn = sa.adversarial_synonym_replacement(sample=sample)

oa = OrderingActions(queries, q_ids)
transformed_queries_shuffled_order = oa.shuffle_word_order(sample=sample)

ma = MispellingActions(queries, q_ids)
transformed_queries_mispelling = ma.mispelling_chars(sample=sample)


transformed_queries =  transformed_queries_mispelling +\
  transformed_queries_shuffled_order  +\
  transformed_queries_syn + \
  transformed_queries_paraphrase_models + \
  transformed_queries_back_translation +  \
  transformed_queries_stop_word_removal +  \
  transformed_queries_summarizer# #+ \
  #transformed_queries_trec_desc_to_title

transformed_queries = pd.DataFrame(transformed_queries, columns = ["q_id", "original_query", "variation", "method", "transformation_type"])

[INFO] Please confirm you agree to the authors' data usage agreement found at <https://ciir.cs.umass.edu/downloads/Antique/readme.txt>
[INFO] [starting] https://ciir.cs.umass.edu/downloads/Antique/antique-train-queries.txt
[INFO] [finished] https://ciir.cs.umass.edu/downloads/Antique/antique-train-queries.txt: [00:00] [137kB] [1.60MB/s]


You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thouroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


 77%|███████▋  | 10/13 [00:57<00:16,  5.65s/it]/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
100%|██████████| 200/200 [02:50<00:00,  1.18it/s]


/usr/local/lib/python3.10/dist-packages/transformers/models/t5/tokenization_t5_fast.py:160: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-base automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(
100%|██████████| 13/13 [00:32<00:00,  2.47s/it]
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
100%|██████████| 200/200 [00:00<00:00, 412.05it/s]


In [ ]:
transformed_queries

,q_id,original_query,variation,method,transformation_type
0,1907320,how do i get college money?,how do i gte college money?,WordSwapNeighboringCharacterSwap,mispelling
1,1907320,how do i get college money?,how do i get colKege money?,WordSwapRandomCharacterSubstitution,mispelling
2,1907320,how do i get college money?,how do i get college moneu?,WordSwapQWERTY,mispelling
3,3884772,how about a bumper sticker thats says:?,how about a ubmper sticker thats says:?,WordSwapNeighboringCharacterSwap,mispelling
4,3884772,how about a bumper sticker thats says:?,how about a bumper stiuker thats says:?,WordSwapRandomCharacterSubstitution,mispelling
...,...,...,...,...,...
1995,993601,why were black women abused sexually after the...,black women abused sexually after,summarization_with_t5-base,naturality
1996,4069320,athiests: how did you find the light?,athiests: how,summarization_with_t5-base,naturality
1997,2897078,how to get rid of stript throat?,stript throat: how to get,summarization_with_t5-base,naturality
1998,1034050,why does the earth keep getting hotter and hot...,: why does the earth keep,summarization_with_t5-base,naturality


###Downloading manually annoted author query variation generations
Reading and downloading ccessing author provided manuallly quality checked query variations

In [ ]:
# functions to download files from google drive. Copied from G. Penha repo
import requests
import zipfile


def save_response_content(response, destination):
    CHUNK_SIZE = 32768

    with open(destination, "wb") as f:
        for chunk in response.iter_content(CHUNK_SIZE):
            if chunk: # filter out keep-alive new chunks
                f.write(chunk)

def get_confirm_token(response):
    for key, value in response.cookies.items():
        if key.startswith('download_warning'):
            return value

    return None

def download_file_from_google_drive(id, destination):
    URL = "https://docs.google.com/uc?export=download"

    session = requests.Session()

    response = session.get(URL, params = { 'id' : id }, stream = True)
    token = get_confirm_token(response)

    if token:
        params = { 'id' : id, 'confirm' : token }
        response = session.get(URL, params = params, stream = True)

    save_response_content(response, destination)

In [ ]:
#Downloading annotated query variations
download_file_from_google_drive('1YQCuO6XrGzWyNyidV1fNL2zZ4BkQ9SAv', "variations_trec2019_labeled.csv")
download_file_from_google_drive('1tZY8QPuhQKihWR9CSYm-6KcJ3Xtc5kkh', "variations_antique_labeled.csv")

In [ ]:
# import locale
# def getpreferredencoding(do_setlocale = True):
#     return "UTF-8"
# locale.getpreferredencoding = getpreferredencoding
# !python /content/query_variation_generators/disentangled_information_needs/evaluation/query_rewriting.py \
#         --task 'irds:msmarco-passage/trec-dl-2019/judged' \
#         --output_dir $OUT_DIR/ \
#         --variations_file $OUT_DIR/$VARIATIONS_FILE_TREC_DL \
#         --retrieval_model_name "BM25+KNRM" \
#         --train_dataset "irds:msmarco-passage/train" \
#         --max_iter 50

#Reproducing Main Table
###Downloading Files from Public Repo
Requires creation of download_and_extract_from_google_drive function and link input to download each file, including indexes for marco and antiques

In [ ]:
with zipfile.ZipFile("/content/gdrive/MyDrive/msmarco-index.zip","r") as zip_ref:
        zip_ref.extractall("./")
with zipfile.ZipFile('/content/gdrive/MyDrive/antique-index.zip', 'r') as zip_ref:
        zip_ref.extractall('./')

In [ ]:
# Main table for TREC-DL (do the same for Antique if needed)
#BM25
# !python /content/query_variation_generators/disentangled_information_needs/evaluation/query_rewriting.py --task 'irds:msmarco-passage/trec-dl-2019/judged' \
#         --output_dir './' \
#         --variations_file variations_trec2019_labeled.csv

# !python /content/query_variation_generators/disentangled_information_needs/evaluation/query_rewriting.py --task 'irds:msmarco-passage/trec-dl-2019/judged' \
#          --output_dir './' \
#          --variations_file variations_trec2019_labeled.csv \
#          --retrieval_model_name "BM25+RM3"

# !python /content/query_variation_generators/disentangled_information_needs/evaluation/query_rewriting.py --task 'irds:msmarco-passage/trec-dl-2019/judged' \
#         --output_dir './' \
#          --variations_file variations_trec2019_labeled.csv \
#         --retrieval_model_name "BM25+BERT" \
#         --train_dataset "irds:msmarco-passage/train" \
#         --max_iter 801

# !python /content/query_variation_generators/disentangled_information_needs/evaluation/query_rewriting.py --task 'irds:msmarco-passage/trec-dl-2019/judged' \
#         --output_dir './' \
#          --variations_file variations_trec2019_labeled.csv \
#         --retrieval_model_name "BM25+KNRM" \
#         --train_dataset "irds:msmarco-passage/train" \
#         --max_iter 400

# !python /content/query_variation_generators/disentangled_information_needs/evaluation/query_rewriting.py --task 'irds:msmarco-passage/trec-dl-2019/judged' \
#         --output_dir './' \
#          --variations_file variations_trec2019_labeled.csv \
#         --retrieval_model_name "BM25+T5"

# !python /content/query_variation_generators/disentangled_information_needs/evaluation/query_rewriting.py --task 'irds:msmarco-passage/trec-dl-2019/judged' \
#         --output_dir './' \
#          --variations_file variations_trec2019_labeled.csv \
#         --retrieval_model_name "https://macavaney.us/pt_models/msmarco.epic.seed42.tar.gz"

# !python /content/query_variation_generators/disentangled_information_needs/evaluation/query_rewriting.py --task 'irds:msmarco-passage/trec-dl-2019/judged' \
#         --output_dir './' \
#          --variations_file variations_trec2019_labeled.csv \
#         --retrieval_model_name "https://macavaney.us/pt_models/msmarco.convknrm.seed42.tar.gz"

In [ ]:
!python /content/query_variation_generators/disentangled_information_needs/evaluation/query_rewriting.py --task 'irds:msmarco-passage/trec-dl-2019/judged' \
        --output_dir './' \
         --variations_file variations_trec2019_labeled.csv \
        --retrieval_model_name "BM25+BERT" \
        --train_dataset "irds:msmarco-passage/train" \
        --max_iter 801

terrier-assemblies 5.8 jar-with-dependencies not found, downloading to /root/.pyterrier...
Done
terrier-python-helper 0.0.8 jar not found, downloading to /root/.pyterrier...
Done
terrier-prf -SNAPSHOT jar not found, downloading to /root/.pyterrier...
Done
PyTerrier 0.10.0 has loaded Terrier 5.8 (built by craigm on 2023-11-01 18:05) and terrier-helper 0.0.8

No etc/terrier.properties, using terrier.default.properties for bootstrap configuration.
Better speed can be achieved with apex installed from https://www.github.com/nvidia/apex.
21:46:53.603 [main] WARN org.terrier.structures.FSADocumentIndex - This index has fields, but FSADocumentIndex is used (which stores fields lengths on disk); If using field-based models such as BM25F, change to index.document.class in the index  properties file to FSAFieldDocumentIndex or FSADocumentIndexInMemFields to support efficient retrieval. If you don't use (e.g.) BM25F, this warning can be ignored
2023-11-05 21:46:57,084 [INFO] Fitting BERT.
config 

In [ ]:
!cat /content/query_variation_generators/query_rewriting_irds:msmarco-passage-trec-dl-2019-judged_model_BM25.csv

In [ ]:
!python /content/query_variation_generators/disentangled_information_needs/evaluation/query_rewriting.py --task 'irds:msmarco-passage/trec-dl-2019/judged' \
        --output_dir './' \
        --variations_file variations_trec2019_labeled.csv

In [ ]:
!ls